
## Tweets

A tweet consists of many data fields. [Here is an example](https://gist.github.com/arapat/03d02c9b327e6ff3f6c3c5c602eeaf8b). You can learn all about them in the Twitter API doc. We are going to briefly introduce only the data fields that will be used in this homework.

* `created_at`: Posted time of this tweet (time zone is included)
* `id_str`: Tweet ID - we recommend using `id_str` over using `id` as Tweet IDs, becauase `id` is an integer and may bring some overflow problems.
* `text`: Tweet content
* `user`: A JSON object for information about the author of the tweet
    * `id_str`: User ID
    * `name`: User name (may contain spaces)
    * `screen_name`: User screen name (no spaces)
* `retweeted_status`: A JSON object for information about the retweeted tweet (i.e. this tweet is not original but retweeteed some other tweet)
    * All data fields of a tweet except `retweeted_status`
* `entities`: A JSON object for all entities in this tweet
    * `hashtags`: An array for all the hashtags that are mentioned in this tweet
    * `urls`: An array for all the URLs that are mentioned in this tweet


## Data source

All tweets are collected using the [Twitter Streaming API](https://dev.twitter.com/streaming/overview).


# Part 0: Load data to a RDD

In [19]:
#%pylab inline
import numpy as np

In [20]:
# Your code here
# (Nishant:  add code to take one time sliced in one time and delete after use)


count =0
with open('./Data/sliced.txt') as f:
    for l in f.readlines():
        count = count+1
        print count;
        if len(l.strip()) is not 0:
            file1 = l.strip()
            print file1
            if count == 1:
                doc1 = sc.textFile(file1).cache()
            if count == 2:
                doc2 = sc.textFile(file1).cache()
            if count == 3:
                doc3 = sc.textFile(file1).cache()
            if count == 4:
                doc4 = sc.textFile(file1).cache()
        # if count == 5:
            #    doc5 = sc.textFile(file1).cache()
    
print doc1.count()
print doc2.count()
print doc3.count()
print doc4.count()
#print doc5.count()

1
./Data/t1/1.txt
2
./Data/t1/2.txt
3
./Data/t1/3.txt
4
./Data/t1/4.txt
2466
1325
609
1048


# Part 1: Parse JSON strings to JSON objects

Python has built-in support for JSON.

**UPDATE:** Python built-in json library is too slow. In our experiment, 70% of the total running time is spent on parsing tweets. Therefore we recommend using [ujson](https://pypi.python.org/pypi/ujson) instead of json. It is at least 15x faster than the built-in json library according to our tests.

## Broken tweets and irrelevant messages

The data of this assignment may contain broken tweets (invalid JSON strings). So make sure that your code is robust for such cases.

In addition, some lines in the input file might not be tweets, but messages that the Twitter server sent to the developer (such as [limit notices](https://dev.twitter.com/streaming/overview/messages-types#limit_notices)). Your program should also ignore these messages.

*Hint:* [Catch the ValueError](http://stackoverflow.com/questions/11294535/verify-if-a-string-is-json-in-python)


(1) Parse raw JSON tweets to obtain valid JSON objects. From all valid tweets, construct a pair RDD of `(user_id, text)`, where `user_id` is the `id_str` data field of the `user` dictionary (read [Tweets](#Tweets) section above), `text` is the `text` data field.

In [21]:
import ujson

def safe_parse(x):
    try:
        json_object = ujson.loads(x)
    except ValueError, e:
        pass # invalid json
    else:
        return json_object


In [22]:
import re
regex = re.compile('[%s]' % re.escape('!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~\n'))

In [23]:
doc1text = doc1.filter(lambda x: 'user' in x and 'id_str' in x and 'text' in x).map(safe_parse)\
        .map(lambda x: regex.sub('', x["text"].lower()).encode('utf-8')).cache()
doc2text = doc2.filter(lambda x: 'user' in x and 'id_str' in x and 'text' in x).map(safe_parse)\
        .map(lambda x: regex.sub('', x["text"].lower()).encode('utf-8')).cache()
doc3text = doc3.filter(lambda x: 'user' in x and 'id_str' in x and 'text' in x).map(safe_parse)\
        .map(lambda x: regex.sub('', x["text"].lower()).encode('utf-8')).cache()
doc4text = doc4.filter(lambda x: 'user' in x and 'id_str' in x and 'text' in x).map(safe_parse)\
        .map(lambda x: regex.sub('', x["text"].lower()).encode('utf-8')).cache()
#doc5text = doc5.filter(lambda x: 'user' in x and 'id_str' in x and 'text' in x).map(safe_parse)\
 #       .map(lambda x: regex.sub('', x["text"].lower()).encode('utf-8')).cache()

# Remove Punctuation Stopwords And Strip

In [24]:
print doc1text.first()
print doc2text.first()
print doc3text.first()
print doc4text.first()
#print doc5text.first()

#alwaystrump  trump2016 #trumptrain #makeamericagreatagain vote trump  httpstcosadz1nys5o
i donated @berniesanders #feelthebern #notmeus httpstcoqssmygivwd
@carminezozzora @jaredwyand @hillaryclinton she is divisive there will be no coming together under #hillary she refers to gop as the enemy
mt @texasshebandit a vote for #tedcruz is a vote for conservatism httpstcobfhdqvivvw #cruzcrew #pjnet


In [25]:
## removing punctuation stopwords and https
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
stop_words =set(stopwords.words("english"))
stop_words|=set(["edu", "com", "also", "still", "anyone", "cc" , "ca", "us", "much", "even", "would", "see", "rt", 'is', 'of'])
st = PorterStemmer()

#IMP: Put yout nltk_data folder in /usr/local/share for it to work :)

def getwords(tweet):
    return [st.stem(x) for x in tweet if not ( x.startswith('htt') or x.startswith('@') or x in stop_words or x is ' ')]

punc = string.punctuation+"\n"

tweetNeat1 = doc1text.map(lambda s: getwords(s.decode('utf-8').split(" "))).filter(lambda x: len(x)>0).map(lambda x: filter(None,x))
tweetNeat2 = doc2text.map(lambda s: getwords(s.decode('utf-8').split(" "))).filter(lambda x: len(x)>0).map(lambda x: filter(None,x))
tweetNeat3 = doc3text.map(lambda s: getwords(s.decode('utf-8').split(" "))).filter(lambda x: len(x)>0).map(lambda x: filter(None,x))
tweetNeat4 = doc4text.map(lambda s: getwords(s.decode('utf-8').split(" "))).filter(lambda x: len(x)>0).map(lambda x: filter(None,x))
#tweetNeat5 = doc5text.map(lambda s: getwords(s.decode('utf-8').split(" "))).filter(lambda x: len(x)>0).map(lambda x: filter(None,x))

# Make Dictionary of Words

In [26]:
## make dictionary 
fulltweet = tweetNeat1+tweetNeat2+tweetNeat3+tweetNeat4
#+tweetNeat5
vocab = fulltweet.flatMap(lambda x: x).distinct().collect()

V= len(vocab)
print V


4973


# Documents from tweets

In [27]:
tweetList = 4*[[]]

tweetList[0] = tweetNeat1.flatMap(lambda x:x).collect()
tweetList[1] = tweetNeat2.flatMap(lambda x:x).collect()
tweetList[2] = tweetNeat3.flatMap(lambda x:x).collect()
tweetList[3] = tweetNeat4.flatMap(lambda x:x).collect()
#tweetList[4] = tweetNeat5.flatMap(lambda x:x).collect()

D = len(tweetList)
D

4

In [28]:
tweetList[0]

[u'#alwaystrump',
 u'trump2016',
 u'#trumptrain',
 u'#makeamericagreatagain',
 u'vote',
 u'trump',
 u'#trump',
 u'#trump2016',
 u'sen',
 u'jeff',
 u'session',
 u'endors',
 u'donald',
 u'trump',
 u'isnt',
 u'campaign',
 u'movement',
 u'#trump2016',
 u'#alprimari',
 u'donald',
 u'j',
 u'trump',
 u'49rubio',
 u'cruz',
 u'carson',
 u'kasich',
 u'47#makeamericagreatagain',
 u'#trump2016',
 u'#trumptrain\U0001f682\U0001f1fa\U0001f1f8\U0001f1fa\U0001f1f8\U0001f1fa\U0001f1f8\U0001f1fa\U0001f1f8\U0001f1fa\U0001f1f8\U0001f1fa\U0001f1f8',
 u'silent',
 u'major',
 u'stand',
 u'#trump',
 u'battl',
 u'msm',
 u'#alwaystrump',
 u'#neverrubio',
 u'#nevercruz',
 u'bill',
 u'prevent',
 u'sanctuari',
 u'citi',
 u'florida',
 u'die',
 u'marco',
 u'rubio',
 u'watch',
 u'#rubio',
 u'#alwaystru\u2026',
 u'corrupt',
 u'incarn',
 u'#trump2016',
 u'#secprimari',
 u'former',
 u'ted',
 u'cruz',
 u'support',
 u'vote',
 u'donald',
 u'trump',
 u'#supertuesday',
 u'trust',
 u'trump',
 u'#trumptrain',
 u'#alwaystrump',
 

# Part 2 : Collapsed Gibbs Sampler for Dynamic Topic Model
Ref : 
1. Mimno (collapsed gibbs) http://dirichlet.net/pdf/mimno08gibbs.pdf
2. Mimno (CG for logistic) http://psiexp.ss.uci.edu/research/papers/sciencetopics.pdf
3. Blei (Dynamic Topic Modelling) http://www.stat.uchicago.edu/~lafferty/pdf/dtm.pdf


In [29]:
import random
from scipy.stats import truncnorm 

def sampleFromDiscrete(probs):
    temp = random.random()
    total =0
    for i in range(len(probs)):
        total+=probs[i]
        if(temp<total):
            return i
    return 0


Initialization of hyperparameters

In [48]:
## Annotate words
import numpy as np

tweetHist0 = sc.parallelize(tweetList[0]).map(lambda w: vocab.index(w)).collect()
tweetHist1 = sc.parallelize(tweetList[1]).map(lambda w: vocab.index(w)).collect()
tweetHist2 = sc.parallelize(tweetList[2]).map(lambda w: vocab.index(w)).collect()
tweetHist3 = sc.parallelize(tweetList[3]).map(lambda w: vocab.index(w)).collect()

tweetHist = sc.parallelize([tweetHist0, tweetHist1, tweetHist2, tweetHist3])\
              .map(lambda y: np.bincount(y, minlength=len(vocab)))

    
N = tweetHist.count()
N

4

In [52]:
K=15
alpha = np.ones(V)*0.1 #hyperparameter
A = [alpha]*K
beta = np.ones(K)
pi = np.random.dirichlet(beta)
pi1=pi
theta = np.random.dirichlet(alpha, K)
no_tweets = N
doc_vec = tweetHist.collect()
docs = set(range(0, N))
z = [0]*N
for i in range(K):
    d = docs.pop()
    z[d] = i
    dist = np.linalg.norm(doc_vec - doc_vec[d], axis=1)
    sorted_dist = np.argsort(dist)[0:N/K]
    for k in sorted_dist:
        docs.discard(k)
        doc_vec[k] = [-inf]*len(doc_vec[0])
        z[k] = i

KeyError: 'pop from an empty set'

# Naive Bayes

In [18]:
doc_vectors = tweetHist.collect()
for i in range(200):
    print bincount(z)
    pi = np.random.dirichlet(betas + bincount(z, minlength=K))
    sum_z = np.zeros((K, V))
    for i in range(no_tweets):
        sum_z[z[i]]+=doc_vectors[i]
    for i in range(K):
        theta[i] = np.random.dirichlet(A[i]+sum_z[i])
    for d in range(no_tweets):
        for i in range(K):
            pi1[i]= np.dot(np.log(theta[i]),doc_vectors[d])
        pi1 = np.exp(pi1-np.max(pi1))
        for i in range(K):
            pi1[i] *= pi[i]
        pi1 = pi1/np.sum(pi1)
        #print pi1
        z[d]=sampleFromDiscrete(pi1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [ ]:
print z

In [19]:
for i in range(D):
    print np.bincount(z[i])

[1314 1561 1542 1423 1598 1518 1449 1404 1454 1468 1476 1284 1488 1464 1338]
[700 761 738 790 701 683 749 688 672 702 627 676 750 803 702]
[381 314 365 322 341 376 397 315 360 323 342 360 359 344 353]
[586 613 617 589 571 653 531 642 492 535 599 628 655 701 571]


In [ ]:
print len(theta)
indices = argsort(theta, axis = 1)
best = indices[:,-40:]
for k in freq_class:
    wordlist = [vocab[best[k][j]] for j in range(len(best[0]))]          
    print wordlist